In [3]:
import pandas as pd
import sqlalchemy as sql
import numpy as np
import math

In [4]:
# INDEX TICKER NOT OPTION TICKER. SPX not SPXW for instance
symbol = "RUT"
horizontal_skew_symbol = "RUTW"

In [5]:
engine = sql.create_engine("postgresql://postgres@localhost/optiondata")
# Select all daily data for given symbol
data = pd.read_sql("SELECT * FROM underlying_data WHERE symbol='{0}' AND bar_length='1440' AND data_type='TRADES'".format(symbol),
                   engine)
# Squared log returns
returns = np.square(np.log(data['close']).diff())

In [9]:
returns

0               NaN
1      2.268341e-05
2      3.288093e-07
3      9.499357e-05
4      7.537426e-05
5      7.874968e-06
6      5.537764e-05
7      4.183830e-09
8      5.772347e-06
9      5.042693e-05
10     8.309655e-05
11     4.947230e-06
12     2.540853e-06
13     3.102429e-05
14     7.588934e-07
15     2.428256e-05
16     6.505163e-09
17     9.316828e-05
18     1.511530e-06
19     3.693070e-05
20     4.425559e-06
21     9.989505e-04
22     1.842010e-04
23     3.701505e-04
24     3.628046e-07
25     1.602501e-04
26     3.353676e-06
27     3.973631e-05
28     1.162392e-05
29     1.825778e-04
           ...     
222    2.364799e-04
223    4.075598e-05
224    3.520198e-07
225    6.345956e-05
226    2.107809e-05
227    1.878207e-04
228    1.137476e-04
229    2.716395e-05
230    1.053956e-05
231    6.305122e-05
232    8.842723e-07
233    4.886598e-06
234    3.791862e-06
235    7.770046e-06
236    9.734399e-05
237    1.645673e-07
238    2.049509e-05
239    2.376106e-06
240    7.288946e-05


In [8]:
windows = [x+2 for x in range(120)]
results_dict = {}

def vol_calc(array):
    return np.sqrt(np.sum(array) * (1 / (array.size - 1))) * np.sqrt(252)

# Calculates rolling volatility for each window length and gets the max, min, and average. Creates series and adds it to the dict
# indexed by the window length. Creates dataframe from result dict
for window in windows:
    std = returns.rolling(window=window).apply(vol_calc)
    std_max = std.max()
    std_min = std.min()
    std_avg = std.mean()
    result = pd.Series({"std_max": std_max, "std_min": std_min, "std_avg": std_avg})
    results_dict[window] = result
    
vol_cone = pd.DataFrame.from_dict(results_dict, orient="index")

In [14]:
type(vol_calc(returns))

numpy.float64

In [ ]:
# Gets data for past year
recent_data = pd.read_sql("SELECT * FROM underlying_data WHERE "
                          "date > (current_date - INTERVAL '252' DAY) AND symbol='{0}' AND data_type='TRADES'".format(symbol),
                          engine)
# Squared log Returns
recent_returns = np.square(np.log(recent_data['close']).diff())

# Creates 1d array of lookback periods, and creates placeholder frame for vol values corresponding to those lookback periods
lookback = np.arange(2, 121)
recent_vol = np.ones(lookback.size)

# For each lookback, calculate vol and put that value in the corresponding place in the recent_vol array.
for x in lookback:
    # Finds the INDEX position in the lookback array of the lookback value so the volatility can be placed in the corresponding
    # location in the recent_vol frame
    index_pos = np.where(lookback == x)[0]
    vol = np.sqrt(np.sum(recent_returns[-x:]) * (1 / (recent_returns[-x:].size - 1))) * np.sqrt(252)
    recent_vol[index_pos] = vol

In [359]:
# Gets most recent date from horizontal skew table
recent_date = pd.read_sql("SELECT DISTINCT date FROM horizontal_skew ORDER BY date DESC LIMIT 1", engine).loc[0].date
recent_date = recent_date.isoformat()

# Gets info for that date
implied_vol = pd.read_sql("SELECT * FROM horizontal_skew WHERE symbol='{0}' "
                          "AND date='{1}'".format(horizontal_skew_symbol, recent_date), engine)

# Gets days to expiration and ATM volatility as arrays for plotting
skew_dte = implied_vol['dte'].as_matrix()
skew_vol = implied_vol['atm_vol'].as_matrix()

In [ ]:
# Plots calculated vol cone with current vol calculated using the lookback periods on the x axis
vol_cone.plot()
# Plots using lookback array for x values and corresponding recent_vol values for y
plt.plot(lookback, recent_vol)
# Plots actual ATM vol for options with x-axis days to expiration
plt.plot(skew_dte, skew_vol)